# Network analysis of BiciMAD

In [20]:
import pandas as pd
import plotly.express as px
import json
import numpy as np
from scipy import stats
import networkx as nx
import plotly.graph_objects as go
from pyvis.network import Network
from geopy.distance import geodesic
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook
plt.rcParams["figure.figsize"] = [10, 5]

In [21]:
stations = pd.read_csv('data/bases_bicimad.csv', sep=';', index_col=False)
stations.drop(stations.loc[stations['Número'].isin(['001 b',
                                                    '020 ampliacion',
                                                    '025 b',
                                                    '080 b',
                                                    '090 ampliacion',
                                                    '106 b',
                                                    '111 b',
                                                    '116 b',
                                                    '128 ampliacion',
                                                    '140 ampliación',
                                                    '161 ampliacion',
                                                    ])].index, inplace=True)
#stations['Número'].iloc[0] = 1
stations['Número'] = stations['Número'].replace({'001 a': 1,
                                                 '025 a': 25,
                                                 '080 a': 80,
                                                 '106 a': 106,
                                                 '111 a': 111,
                                                 '116 a': 116
                                                 })
stations['Número'] = pd.to_numeric(stations['Número'])


def distance_calc (row):
    start = (40.4166, -3.70384)
    stop = (row['Latitud'], row['Longitud'])

    return geodesic(start, stop).meters

stations['dist_km0'] = stations.apply (lambda row: distance_calc(row), axis=1)

stations

,Número,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Número de Plazas,Longitud,Latitud,Direccion,dist_km0
0,1,"440443,61","4474290,65",43803,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",2,BiciMAD,30,-3.701998,40.417111,"ALCALA, CALLE, DE, 2",166.302566
2,2,"440134,83","4474678,23",41813,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,BiciMAD,24,-3.705674,40.420580,"MIGUEL MOYA, CALLE, DE, 1",468.555831
3,3,"440012,98","4475760,68",41813,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,BiciMAD,18,-3.707212,40.430322,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL, 2",1550.414033
4,4,"440396,4","4475565,36",41813,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,BiciMAD,24,-3.702674,40.428590,"MANUELA MALASAÑA, CALLE, DE, 3",1335.130869
5,5,"440447,06","4475539,6",41813,01 CENTRO,01-04 JUSTICIA,"FUENCARRAL, CALLE, DE",106,BiciMAD,27,-3.702074,40.428362,"FUENCARRAL, CALLE, DE, 106",1314.652786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,257,"438063,25","4476718,06",44194,09 MONCLOA-ARAVACA,09-03 CIUDAD UNIVERSITARIA,"JUAN DE HERRERA, AVENIDA, DE",1,BiciMAD,24,-3.730290,40.438804,"JUAN DE HERRERA, AVENIDA, DE, 1",3334.216323
265,258,"438326,14","4477849,84",44194,09 MONCLOA-ARAVACA,09-03 CIUDAD UNIVERSITARIA,"COMPLUTENSE, AVENIDA",frente al 14,BiciMAD,24,-3.727300,40.449019,"COMPLUTENSE, AVENIDA, frente al 14",4113.642283
266,259,"438344,78","4477338,89",44194,09 MONCLOA-ARAVACA,09-03 CIUDAD UNIVERSITARIA,"RAMON Y CAJAL, PLAZA, DE",S/N,BiciMAD,24,-3.727031,40.444418,"RAMON Y CAJAL, PLAZA, DE, S/N",3662.530936
267,260,"438328,53","4477762,41",44194,09 MONCLOA-ARAVACA,09-03 CIUDAD UNIVERSITARIA,"JOSE ANTONIO NOVAIS, CALLE, DE",S/N,BiciMAD,24,-3.727264,40.448232,"JOSE ANTONIO NOVAIS, CALLE, DE, S/N",4035.819273


In [22]:
trips = pd.read_json('data/201906_Usage_Bicimad.json', lines=True,
                     encoding='latin-1', convert_dates='unplug_hourTime').query(
    'user_type in [1, 2]')[['idunplug_station', 'idplug_station',
                            'travel_time', 'ageRange', 'unplug_hourTime',
                            'zip_code', 'user_type']]

trips['zip_code'] = pd.to_numeric(trips['zip_code'], errors='coerce')
trips['unplug_hourTime'] = pd.to_datetime(trips['unplug_hourTime'].apply(lambda x: x['$date']))
trips = trips[trips['travel_time'] < 60*60*8]
trips['od_time_ratio'] = trips['travel_time'] / \
    trips.groupby(['idunplug_station', 'idplug_station'])[
    'travel_time'].transform('mean')
trips['o_time_ratio'] = trips['travel_time'] / \
    trips.groupby(['idunplug_station'])[
    'travel_time'].transform('mean')
trips['d_time_ratio'] = trips['travel_time'] / \
    trips.groupby(['idplug_station'])[
    'travel_time'].transform('mean')
trips['hour'] = trips['unplug_hourTime'].dt.hour
trips['day_type'] = trips['unplug_hourTime'].dt.weekday.isin([5, 6]).astype(int)
trips = trips.merge(stations[['Número', 'dist_km0']], how='left',
              left_on='idunplug_station', right_on='Número').rename(columns={'dist_km0': 'o_dist_km0'}).drop(columns='Número')
trips = trips.merge(stations[['Número', 'dist_km0']], how='left',
              left_on='idplug_station', right_on='Número').rename(columns={'dist_km0': 'd_dist_km0'}).drop(columns='Número')
trips = trips.fillna(trips.mean())

trips

<ipython-input-22-81e3bbcd28b2>:25: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



,idunplug_station,idplug_station,travel_time,ageRange,unplug_hourTime,zip_code,user_type,od_time_ratio,o_time_ratio,d_time_ratio,hour,day_type,o_dist_km0,d_dist_km0
0,90,66,219,0,2019-06-01 00:00:00+02:00,94845.066194,1,0.243454,0.253708,0.254429,0,1,1949.659347,1971.364279
1,71,136,359,4,2019-06-01 00:00:00+02:00,28039.000000,1,0.493381,0.406542,0.346933,0,1,2531.689730,3371.451753
2,39,38,375,4,2019-06-01 00:00:00+02:00,28013.000000,1,1.232202,0.407593,0.396823,0,1,736.833314,873.752025
3,66,90,264,5,2019-06-01 00:00:00+02:00,28009.000000,1,0.772850,0.270303,0.297862,0,1,1969.964110,1949.659347
4,152,166,367,4,2019-06-01 00:00:00+02:00,28006.000000,1,0.779009,0.390789,0.384688,0,1,4031.870979,858.577165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424492,55,71,471,0,2019-06-26 23:00:00+02:00,94845.066194,1,0.635430,0.514104,0.501796,23,0,845.361722,2531.689730
424493,51,135,317,5,2019-06-26 23:00:00+02:00,28045.000000,1,0.715425,0.356829,0.259280,23,0,1314.521569,2915.287342
424494,172,174,1558,0,2019-06-26 23:00:00+02:00,94845.066194,1,0.903014,1.395094,1.545086,23,0,1953.275690,2140.986664
424495,35,2,2199,0,2019-06-26 23:00:00+02:00,94845.066194,1,2.230676,2.027131,2.112135,23,0,615.794745,468.555831


In [23]:
trips_grouped_count = trips.groupby(['idunplug_station', 'idplug_station']).agg(
    weight=('travel_time', 'count'), cost=('travel_time', 'mean')).reset_index()

In [24]:
trips_grouped_count['cost'] = trips_grouped_count['cost'].astype('int64')

In [25]:
trips_grouped_count[['weight', 'cost']].describe()

,weight,cost
count,28965.000000,28965.000000
mean,14.655515,1059.893112
std,17.390449,739.444925
min,1.000000,95.000000
25%,4.000000,682.000000
50%,9.000000,928.000000
75%,19.000000,1239.000000
max,306.000000,21011.000000


In [26]:
od_df_count = trips.pivot_table(index='idunplug_station',
                                columns='idplug_station',
                                values='travel_time',
                                aggfunc=len,
                                fill_value=0)

In [27]:
px.histogram(trips_grouped_count['weight'], marginal='box')

In [28]:
G = nx.from_pandas_edgelist(trips_grouped_count,
                            source='idunplug_station',
                            target='idplug_station',
                            edge_attr=['weight']
                            )
nx.write_gexf(G, 'bicimad.gexf')

In [29]:
G = nx.from_pandas_edgelist(trips_grouped_count[trips_grouped_count['weight'] > np.percentile(trips_grouped_count['weight'], 99)],
                            source='idunplug_station',
                            target='idplug_station',
                            edge_attr=['weight', 'cost']
                            )

In [30]:
for n, d in G.degree:
    G.nodes[n]['size'] = d
    G.nodes[n]['title'] = stations[stations['Número'] == n]['Direccion'].reset_index(drop=True).get(0, None)

In [31]:
for t in G.edges:
    e = G.edges[t]
    e['title'] = 'Cost: ' + str(round(e['cost']/60, 3)) + ' min.'

In [32]:
print(nx.info(G), '\nDensity: ' + str(round(nx.density(G), 5)))

Name: 
Type: Graph
Number of nodes: 92
Number of edges: 212
Average degree:   4.6087 
Density: 0.05065


In [33]:
[(n, d, G.nodes[n]['title']) for n, d in sorted(G.degree, key=lambda x: x[1], reverse=True)][:10]

[(149, 21, 'LA HABANA, PASEO, DE, 42'),
 (175, 19, 'JAIME EL CONQUISTADOR, CALLE, DE, 30'),
 (163, 19, 'ESPERANZA, PASEO, DE LA, 0,0833333333333333'),
 (135, 18, 'SAN JUAN DE LA CRUZ, PLAZA, DE, S/N'),
 (57, 17, 'SANTA CRUZ DE MARCENADO, CALLE, DE, 24'),
 (129, 14, 'JOSE ABASCAL, CALLE, DE, 33'),
 (90, 13, 'GOYA, CALLE, DE, 18'),
 (42, 11, 'SANTA ISABEL, CALLE, DE, 32'),
 (19, 11, 'PRIM, CALLE, DE, 4'),
 (132, 11, 'SANTA ENGRACIA, CALLE, DE, 168')]

In [34]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

# Assign each to an attribute in your network
nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

In [35]:
[(n, G.degree[n], b, G.nodes[n]['title']) for n, b in sorted(betweenness_dict.items(), key=lambda x: x[1], reverse=True)][:10]

[(19, 11, 0.28774372542507803, 'PRIM, CALLE, DE, 4'),
 (57, 17, 0.21489806552746704, 'SANTA CRUZ DE MARCENADO, CALLE, DE, 24'),
 (163, 19, 0.18844217262144433, 'ESPERANZA, PASEO, DE LA, 0,0833333333333333'),
 (149, 21, 0.18078109940208628, 'LA HABANA, PASEO, DE, 42'),
 (90, 13, 0.17963208507358477, 'GOYA, CALLE, DE, 18'),
 (175, 19, 0.11071695309809834, 'JAIME EL CONQUISTADOR, CALLE, DE, 30'),
 (83, 9, 0.10746118960404677, 'PIO BAROJA, CALLE, DE, 8 B'),
 (135, 18, 0.1040192687149446, 'SAN JUAN DE LA CRUZ, PLAZA, DE, S/N'),
 (132, 11, 0.10114493602429507, 'SANTA ENGRACIA, CALLE, DE, 168'),
 (84, 8, 0.1006356317674999, 'DOCTOR ESQUERDO, CALLE, DEL, 191')]

In [36]:
nt = Network('1000px', '1000px', notebook=True)
# populates the nodes and edges data structures
nt.from_nx(G)
# nt.toggle_physics(False)
# nt.toggle_stabilization(False)
nt.show('bicimad.html')

In [37]:
px.histogram(trips[(trips['idunplug_station'] == 10) & (trips['idplug_station'] == 10)]['travel_time'] / 60, marginal='box')